# Building the next token mask matrix

In [ ]:
import json
import torch
from itertools import chain 
from ataarangi.data import SequenceTokenizer

tokenizer = SequenceTokenizer('../data/worldstate_tokens.txt', '../data/tokens.txt')

class_successor_dict = json.load(open('../data/class_successors.json'))
token_to_class_dict = json.load(open('../data/token_to_class.json'))

class_to_tokens_dict = {k: list(chain.from_iterable([token_to_class_dict[l] for l in v])) for k, v in class_successor_dict.items()}

source_to_target_token_dict = {k2: v1 for k1, v1 in class_to_tokens_dict.items() for k2 in token_to_class_dict[k1]}
source_to_target_index_dict = {tokenizer.token_map[k]: [tokenizer.token_map[item] for item in v] for k, v in source_to_target_token_dict.items()}

tuple_list = [(k, v) for k, vs in source_to_target_index_dict.items() for v in vs]

# Initialize the mask matrix with zeros
mask_matrix = torch.zeros((tokenizer.vocab_size, tokenizer.vocab_size), dtype=torch.float32)

# Set valid transitions based on the tuple list
for src, dest in tuple_list:
    mask_matrix[src, dest] = 1